In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
from scipy import stats
import pickle as pkl
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import cross_val_score
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
with open("../data/iterate/luther_model_data_1.pkl", 'rb') as picklefile:
    sale = pkl.load(picklefile)
    
# potential zipcode filter to NW side
zips_nw = [60611, 60610, 60654, 60642,
           60622, 60647, 60614, 60657,
           60639, 60641, 60630, 60618,
           60613, 60640, 60625, 60660,
           60626, 60659, 60645]

sale_nw = sale[sale['zipcode'].isin(zips_nw)]

In [7]:
"""
build/filter/transform target and features
"""

model_params = ['price','bedrooms','bathrooms','area','median_income','year_built','duration_float']

# filter down to correlation parameters
model = sale[model_params]

#filter out outliers
model = model[(np.abs(stats.zscore(model)) < 3).all(axis=1)]

model['price']=model['price'].apply(np.log10)
model['area']=model['area'].apply(np.log10)

# make data for linear regression
y = model.pop('price').values
X = StandardScaler().fit_transform(model)

In [8]:
"""
set up train test split
"""

# first split out 20% of the data as a validation set
X_training, X_validate, y_training, y_validate = train_test_split(X, y, test_size=0.2)

# now split out another 20% for cross validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#build initial regression model

#setting up as a polynomial but using degree 1, just to have the easy option later
degree = 1
est = make_pipeline(PolynomialFeatures(degree), LinearRegression())
lr = LinearRegression(fit_intercept=True)

scores_R = cross_val_score(est,
                         X_training,
                         y_training,
                         cv=10)#, scoring='neg_mean_squared_error')
scores_RMSE = cross_val_score(est,
                         X_training,
                         y_training,
                         cv=10, scoring='neg_mean_squared_error')
print('mean CV R score = ' + str(np.mean(scores_R)))
#print('mean CV RMSE score = ' + str(np.mean(np.sqrt(-scores_RMSE))))

current_fit = lr.fit(X_training,y_training)
scores_validate = lr.score(X_validate,y_validate)
print('validation R score = ' + str(scores_validate))

mean CV R score = 0.6575516559606418
validation R score = 0.6494677910597023


In [245]:
# fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
# ax1.hist(y,bins=50)
# ax1.set_title('price - linear')
# ax2.hist(np.log10(y),bins=50)
# ax2.set_title('price - log')
# ax2.set_xlim(3,7)

#plt.hist(X[:,3])

0.6347782783606235

[-0.58323914 -0.43747871 -0.57343243 -0.58067498 -0.59214878 -0.59595941
 -0.59134906 -0.57369241 -0.63564025 -0.62767689]
